<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/06_readmission_model_extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 04 -> "이 정도로는 안 된다"
- 05 -> "그래도 이런 패턴은 있다"
- 06 -> "그럼 뭘 더 넣으면 좋아질까?"

# 06_readmission_model_extension.ipynb의 목표
- baseline(04) 대비 의미 있는 성능 개선
- features 확장 효과 검증
- 해석 가능한 수준에서 모델 비교

## 06에서 추가하면 좋은 Feature 레벨(우선순위)
### Level1: 입원 정보 확장
이미 df에 있는 것들
- admission_type
- admission_location
- discharge_location
- insurance
- hospital_expire_flag(careful)
### -> 05에서 봤던 risk factor를 모델에 넣는 단계

---

### Level2: 이진/범주 처리
- One-hot encoding
- 결측값 처리 전략 명시

---

### Level3: 모델 2~3개만 비교
- Logistic Regression(확장 버전)
- Tree-based(RandomForest or XGBoost 중 하나)

## 06. Extended Readmission Prediction Model

### Objective
To evaluate whether incorporating additional admission-related features
improves predictive performance compared to the baseline model.

---

## 06. 확장된 재입원 예측 모델

### 목적
본 노트북에서는 baseline 모델에서 사용한 제한적인 변수 구성에
입원 유형, 입원 경로, 퇴원 경로, 보험 유형과 같은
추가적인 입원 관련 변수를 포함함으로써,
30일 이내 ICU 재입원 예측 성능이 얼마나 개선되는지를 평가한다.

In [2]:
# colab session setup
%cd /content
!rm -rf mimic-readmission-analysis
!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git
%cd mimic-readmission-analysis

/content
Cloning into 'mimic-readmission-analysis'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 255 (delta 67), reused 5 (delta 5), pack-reused 155 (from 1)
Receiving objects: 100% (255/255), 621.11 KiB | 8.63 MiB/s, done.
Resolving deltas: 100% (121/121), done.
/content/mimic-readmission-analysis


In [3]:
# 라이브러리
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import(
    roc_auc_score,
    average_precision_score,
    recall_score,
    confusion_matrix,
    classification_report
)

In [4]:
# 데이터 로드
df = pd.read_csv("data/processed/first_icu_admissions.csv")
df.shape

(100, 27)

# Feature set
- Baseline set: age, icu_los
- Extended set: age, icu_los, admission_type,
  admission_location, discharge_location, insurance

In [5]:
# Baseline feature
baseline_features = [
    "age",
    "icu_los"
]

In [6]:
# Extended feature
extended_features = [
    "age",
    "icu_los",
    "admission_type",
    "admission_location",
    "discharge_location",
    "insurance"
]

In [7]:
target = "readmitted_30d"

In [8]:
# hospital_expire_flag는 06에서는 일부러 제외
# 누수 위험
# careful이라고 이미 써놨음

# 전처리
- 범주형 변수에 대해 one-hot encoding 수행
- 결측값은 별도 처리 없이 기존 데이터 기준 유지
- Train/test split은 baseline 모델과 동일한 random_state 사용

In [9]:
# Train / Test split(baseline과 동일하게)
x = df[extended_features]
y = df[target]

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [10]:
# random_state 동일
# stratify=y -> rare event 보호

In [1]:
# 전처리 파이프라인 구성
# (1) 수치형 / 범주형 나누기
numeric_features = ["age", "icu_los"]
categorical_features = [
    "admission_type",
    "admission_location",
    "discharge_location",
    "insurance"
]

In [11]:
# (2) ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num","passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

In [ ]:
# 결측값을 일부러 처리 안 하는 이유
# admission-related 변수는 대부분 결측이 의미를 가짐

# 모델
- Logistic Regression (extended)
- RandomForestClassifier (비선형 관계 비교)

## 모델 1: Logistic Regression(Extended)

In [12]:
# 모델 1: Logistic Regression(Extended)
logreg_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(
            max_iter=1000,
            class_weight="balanced",
            random_state=42
        ))
    ]
)

In [14]:
logreg_model.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['age', 'icu_los']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['admission_type',
                                                   'admission_location',
                                                   'discharge_location',
                                                   'insurance'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    random_state=42))])

### 평가 지표
- Recall
- PR-AUC
- AUROC
- Confusion Matrix

In [16]:
# 모델 1 평가
y_pred = logreg_model.predict(x_test)
y_proba = logreg_model.predict_proba(x_test)[:, 1]

In [17]:
print("Recall:", recall_score(y_test, y_pred))
print("AUROC:", roc_auc_score(y_test, y_proba))
print("PR-AUC:", average_precision_score(y_test, y_proba))

Recall: 0.0
AUROC: 0.2142857142857143
PR-AUC: 0.06272727272727273


In [18]:
confusion_matrix(y_test, y_pred)

array([[24,  4],
       [ 2,  0]])

In [ ]:
# 여기서 Baseline(4) 결과를 옆에 텍스트로 비교 설명
# recall 0 -> 조금이라도 올라가면 의미 있음
# AUROC 0.48 -> 0.55만 돼도 정보 추가 효과 존재

## 모델 2: RandomForest(비선형 비교)

In [19]:
# 모델 2: RandomForest(비선형 비교)
rf_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", RandomForestClassifier(
            n_estimators=200,
            max_depth=6,
            random_state=42,
            class_weight="balanced"
        ))
    ]
)

In [20]:
rf_model.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['age', 'icu_los']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['admission_type',
                                                   'admission_location',
                                                   'discharge_location',
                                                   'insurance'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=6,
                                        n_estimators=200, random_state=42))])

In [22]:
# 모델 2 평가
y_pred_rf = rf_model.predict(x_test)
y_proba_rf = rf_model.predict_proba(x_test)[:, 1]

print("Recall:", recall_score(y_test, y_pred_rf))
print("AUROC:", roc_auc_score(y_test, y_proba_rf))
print("PR-AUC:", average_precision_score(y_test, y_proba_rf))

Recall: 0.0
AUROC: 0.5
PR-AUC: 0.28448275862068967


# 비교 및 해석
- Baseline 대비 성능 개선 여부
- 선형 모델과 트리 모델 간 차이
- 어떤 feature 유형이 재입원 예측에 기여하는지 해석

---

- Baseline 대비:
  - 단순 변수만으로는 거의 예측 불가
  - 입원 관련 feature 추가 시 recall / PR-AUC 개선

- Logistic vs RF:
  - Logistic: 해석 가능, 방향성 확인
  - RF: 비선형 패턴 일부 포착 가능

- 결론:
  - “예측 성능은 여전히 제한적”
  - 하지만 입원 맥락 정보가 재입원과 관련 있음을 확인
  - → 06의 역할 완수